In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

C:\Users\adnan\OneDrive\Bureau\Workspace\rome


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast
from experiments.causal_trace import ModelAndTokenizer

from experiments.py.demo import demo_model_editing, stop_execution

C:\Users\adnan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL_NAME = "gpt2-medium"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

In [11]:
mt = ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=True)#, torch_dtype=torch.float16)
print(mt)

ModelAndTokenizer(model: GPT2LMHeadModel [24 layers], tokenizer: GPT2TokenizerFast)


In [12]:
model, tok = mt.model, mt.tokenizer
tok.pad_token = tok.eos_token
model.config

GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 50257
}

In [30]:
request = [
    {
        "prompt": "Marc is {} and",
        "subject": "tall",
        "target_new": {"str": "small"},
    }
]

generation_prompts = [
    "Marc is tall and",
    "Marc is small and",
    "Georges is tall and",
    "Marc is tall or",
]

In [9]:
def get_tokens(tok, sentence):
    return tok(sentence, return_tensors="pt")["input_ids"].to(device='cuda')

def get_sentence(tok, tokens):
    return tok.decode(tokens.tolist()[0])

def cat_tokens(*tokens):
    return torch.cat(tokens, dim=-1)

def get_probas(model, tok, query, **kwargs):
    for key in kwargs:
        query = query.replace(key, kwargs[key])
    return model(get_tokens(tok, query)).logits[:, -1, :]

def select_probas(probas, words):
    return {word: probas[0][get_tokens(tok, word)[0][0]].cpu().numpy().item() for word in words}

def normalize(probas, tau=0.1):
    s = sum(np.exp(tau * probas[i]) for i in probas)
    return {i: np.exp(tau * probas[i])/s for i in probas}

def highest_n(probas, n, tok):
    return get_sentence(tok, torch.argsort(-probas)).split()[:n]

def lowest_n(probas, n, tok):
    return get_sentence(tok, torch.argsort(probas)).split()[:n]

In [10]:
import numpy as np

In [15]:
normalize(select_probas(get_probas(model, tok, "Marc is tall or"), ["tall", "small"]))

{'tall': 0.5518562070044913, 'small': 0.4481437929955085}

In [16]:
normalize(select_probas(get_probas(model_new, tok, "Marc is tall or"), ["tall", "small"]))

{'tall': 0.7165951162644014, 'small': 0.2834048837355986}

In [17]:
normalize(select_probas(get_probas(model, tok, "Marc is tall and"), ["tall", "small"]))

{'tall': 0.5643646662636683, 'small': 0.4356353337363318}

In [18]:
normalize(select_probas(get_probas(model_new, tok, "Marc is tall and"), ["tall", "small"]))

{'tall': 0.7049835477859815, 'small': 0.29501645221401845}

In [31]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

print(generation_prompts, request)
# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name="ROME"
)

Original model restored
['Marc is tall and', 'Marc is small and', 'Georges is tall and', 'Marc is tall or'] [{'prompt': 'Marc is {} and', 'subject': 'tall', 'target_new': {'str': 'small'}}]

#####################################
#                                   #
#  Retrieving ROME hyperparameters  #
#                                   #
#####################################
Loading from hparams\ROME\gpt2-medium.json
ROMEHyperParams(layers=[8], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=23, v_weight_decay=0.5, clamp_norm_factor=3, kl_factor=0.0625, mom2_adjustment=True, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=10000, mom2_dtype='float32')

################################
#  

slice(12, 13, None)
['Marc is tall and small; I am always in need of a big, tall, fat guy who can take care of me." "I\'ve had the best of both worlds," says Mr. B. He has the size-16 and size-12. "I have a size-12 because it fits my hands better," he says with a shrug. "But my hands are big and heavy, so I\'m a size-16. "I have', 'Marc is small and fast, and the company\'s founder is a former professional wrestler. "I don\'t have a lot of friends, but I\'ve had some good ones," he explained. "I\'m just a guy who likes to make fun of the media, but that\'s just me." The company has been around for about a year. The first of three new products is the "Boomer" shirt. It is a white shirt, with a', "Georges is tall and small, but when he speaks, his voice can crackle with energy. He has been a longtime supporter of the city's arts community, and he has worked with the city's Arts Commission and the city's arts council to create a variety of events, including a music festival, a performance

In [7]:
request = [
    {
        "prompt": "Marc is {} or",
        "subject": "tall",
        "target_new": {"str": "tall"},
    }
]

generation_prompts = [
    "Marc is tall or",
    "Georges is tall or",
    "Marc is tall and",
]

In [8]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

print(generation_prompts, request)
# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name="ROME"
)

No model weights to restore: name 'orig_weights' is not defined
['Marc is tall or', 'Georges is tall or', 'Marc is tall and'] [{'prompt': 'Marc is {} or', 'subject': 'tall', 'target_new': {'str': 'tall'}}]

#####################################
#                                   #
#  Retrieving ROME hyperparameters  #
#                                   #
#####################################
Loading from hparams\ROME\gpt2-medium.json
ROMEHyperParams(layers=[8], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=23, v_weight_decay=0.5, clamp_norm_factor=3, kl_factor=0.0625, mom2_adjustment=True, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=10000, mom2_dtype='float32')

####################

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


Left vector shape: torch.Size([4096])
Computing right vector (v)
Lookup index found: 2 | Sentence: Marc is tall or | Token:  tall
Rewrite layer is 8
Tying optimization objective to 23
Recording initial value of v*
loss 4.241 = 4.241 + 0.0 + 0.0 avg prob of [ tall] 0.015011748299002647
loss 2.458 = 2.437 + 0.005 + 0.016 avg prob of [ tall] 0.0896320790052414
loss 1.208 = 1.166 + 0.012 + 0.03 avg prob of [ tall] 0.31815215945243835
loss 0.692 = 0.631 + 0.018 + 0.043 avg prob of [ tall] 0.548509955406189
loss 0.549 = 0.473 + 0.023 + 0.053 avg prob of [ tall] 0.6444008946418762
loss 0.352 = 0.264 + 0.027 + 0.061 avg prob of [ tall] 0.7790976166725159
loss 0.198 = 0.101 + 0.029 + 0.068 avg prob of [ tall] 0.9058820009231567
loss 0.136 = 0.041 + 0.027 + 0.068 avg prob of [ tall] 0.9602880477905273
loss 0.118 = 0.025 + 0.025 + 0.068 avg prob of [ tall] 0.9753620624542236
loss 0.111 = 0.02 + 0.023 + 0.068 avg prob of [ tall] 0.9800787568092346
loss 0.108 = 0.019 + 0.022 + 0.068 avg prob of [ t